# Proyecto Álgebra Conmutativa: Calculando certificados de no viabilidad de problemas combinatorios a través del Nullstellensatz de Hilbert

#### Integrantes: Federico Gálvez Zuleta, Francisco Javier Díaz Perdomo  y Juan Camilo González Cabrera

In [1]:
using Oscar

### 1. Importar el grafo
Primero, importamos el grafo con el que vamos a trabajar. Para esto, definimos las siguientes funciones:

In [2]:
function text_reader(name)
    f = open(name)
    lines = readlines(f)
    E = []
    v= 0
    for l in lines
        x = l[1]
        if x == 'p'
            t = split(l)
            v = parse(Int64, t[3])
        elseif x == 'e'
            t = split(l)
            push!(E, [parse(Int64,t[2]), parse(Int64,t[3])]) 
        end
    end
return v,E
end

function polisGrafo(g,k,R)
    # Devuelve dado un grafo y un entero k, un sistema de polinomios que ayuda a determinar que el grafo es k-colorable Y el anillo de polinomios
    ps = [R[j]^k-1 for j in 1:nv(g)] # los polinomios x^k_i- 1 = 0, ∀i ∈ V(g)
    ps2 = [R[src(r)]^(k-1-l) * R[dst(r)]^(l) for r in collect(edges(g)), l in 0:k-1] # los polinomios sum_l=0^{k-1} x_i^{k-1-l} x_j^l=0 ∀{i, j} ∈ E(G)
    p2 = sum(ps2,dims=2)
    polis = vec(vcat(ps,p2)) # crea una lista con los polinomios
    return polis # retorna la lista de polinomios 
end
        

polisGrafo (generic function with 1 method)

### 2. Certificado de Nullstellensatz

Primero, definimos unas funciones que serán útiles más adelante.

In [3]:
#  Calcula las particiones de d en sumas de n numeros, pueden ser 0
function partition(n::Int, d::Int)
    if n == 1
        return [[d]]
    end
    partitions = []
    for i in 0:d
        for p in partition(n-1, d-i)
            push!(partitions, [i; p])
        end
    end
    return partitions
end
# Calcula los polinomios beta_i sin coeficientes
function polysBi(n::Int, d::Int,genes)
    v = []
    
    for j in 0:d
        v = vcat(v, [reduce(*,[genes[i]^par[i] for i in 1:length(par)]) for par in partition(n,j)])
    end
    return v
end
# Retorna los polinomios beta_i * f_i sin los coeficientes
function creaCertv(d::Int,listap)
    v=[]
    n=length(gens(parent(listap[1])))
    for f in listap
        dd = total_degree(f)
        v = vcat(v,[ f * bi for bi in polysBi(n, d - dd,gens(parent(listap[1])))])
    end
    return v
end

# Calcula el numero de coefficientes
ncoefi(n::Int,e::Int,d::Int,k::Int) = sum(fill(binomial(n+d-k,d-k), n)) + sum(fill(binomial(n + d -k +1,d-k+1), e))::Int

# Pasar a entero
aEntero(a) = (a == GF(2)(1)) ? 1 : (a == GF(2)(0)) ? 0 : nothing
     

aEntero (generic function with 1 method)

Ahora, definimos el algoritmo NullA sobre el campo $\mathbb{F}_2$. Este retorna el certificado de Nullstellensatz que buscamos.

In [4]:
function nulla(g::Graph,cota::Int,k::Int)::Vector{Float64}
    n = nv(g)
    e = ne(g)
    d = 3
    while d <= cota
        nUnkows = ncoefi(n,e,d,k)
        GF2 = GF(2)
        T, y =  PolynomialRing(GF2,["y$i" for i in 1:nUnkows])
        R2, z = PolynomialRing(T,["z$i" for i in 1:n])

        PG = polisGrafo(g, k, R2)

        CERTsinC2 = creaCertv(d,PG)
        CERTv = [ y[i]*CERTsinC2[i] for i in  1:length(CERTsinC2)]

        CERT = reduce(vcat,[collect(terms(f)) for f in CERTv])

        CERT1 = sum(CERT)
        CERTt = collect(terms(CERT1))
        println(CERTt)
        # Creacion del sistema lineal 
        A = []
        for cee in CERTt
            achiquita = vec([aEntero(coeff(cc, yi)) for yi in gens(T), cc in collect(coefficients(cee))])
            if isempty(A)
                A = achiquita
            else
                A=hcat(A,achiquita)
            end
        end
        A = transpose( A)

        b = zeros(Int,length(CERTt))

        b[end]+=1
        sol = A\b
        if any(sol .!= 0) # si no es todo 0, el sistema es consistente
            println("The system of equations F is infeasible")
            return sol
        end
        println("Una iteracion")
        d += 1
    end
    println("The system of equations F is feasible.")
    return true
end

nulla (generic function with 1 method)

Ahora, importamos un grafo básico (Grafo $K_4$ completo) y vemos si es $3$-coloreable.

In [5]:
nver, E = 4, [[1, 2], [1, 3], [1, 4], [2, 3], [2, 4], [3, 4]]
g = Graph{Undirected}(nver)
for v in E
    add_edge!(g, v[1], v[2])
end

In [6]:
nulla(g,56, 3)

AbstractAlgebra.Generic.MPoly{fpMPolyRingElem}[

(y1 + y9 + y14 + y24)*z1^3, (y8 + y9 + y13 + y23)*z1^2*z2, (y8 + y9 + y19 + y29)*z1*z2^2, (y2 + y8 + y18 + y28)*z2^3, (y7 + y12 + y14 + y22)*z1^2*z3, (y7 + y13 + y19)*z1*z2*z3, (y7 + y17 + y18 + y27)*z2^2*z3, (y12 + y14 + y19 + y34)*z1*z3^2, (y13 + y17 + y18 + y33)*z2*z3^2, (y3 + y12 + y17 + y32)*z3^3, (y6 + y11 + y21 + y24)*z1^2*z4, (y6 + y23 + y29)*z1*z2*z4, (y6 + y16 + y26 + y28)*z2^2*z4, (y11 + y22 + y34)*z1*z3*z4, (y16 + y27 + y33)*z2*z3*z4, (y11 + y16 + y31 + y32)*z3^2*z4, (y21 + y24 + y29 + y34)*z1*z4^2, (y23 + y26 + y28 + y33)*z2*z4^2, (y22 + y27 + y31 + y32)*z3*z4^2, (y4 + y21 + y26 + y31)*z4^3, (y5 + y10 + y20)*z1^2, y5*z1*z2, (y5 + y15 + y25)*z2^2, y10*z1*z3, y15*z2*z3, (y10 + y15 + y30)*z3^2, y20*z1*z4, y25*z2*z4, y30*z3*z4, (y20 + y25 + y30)*z4^2, y1 + y2 + y3 + y4]
The system of equations F is infeasible

34-element Vector{Float64}:
  0.22916666666666657
  0.2291666666666664
  0.2291666666666666
  0.22916666666666657
  1.393380451513459e-17
  0.027777777777777728
  0.027777777777777832
 -0.04861111111111104
 -0.04861111111111108
 -4.500139929349619e-18
  ⋮
 -0.04861111111111112
  0.027777777777777707
 -0.048611111111111105
  0.027777777777777873
  1.3933804515134587e-17
 -0.04861111111111107
 -0.04861111111111108
  0.02777777777777777
  0.027777777777777762

Ahora, importamos un grafo arbitrario de la carpeta "Grafos" que tenemos y construimos el sistema de polinomios asociado al $k$-coloreo del grafo.

In [7]:
nver, E = text_reader("Grafos/johnson8-2-4.clq.txt")
g = Graph{Undirected}(nver)
for v in E
    add_edge!(g, v[1], v[2])
end

In [8]:
nulla(g,56, 3)

AbstractAlgebra.Generic.MPoly{fpMPolyRingElem}[(y1 + y115 + y318 + y405 + y840 + y1014 + y1188 + y1942 + y2232 + y2522 + y2812 + y3972 + y4407 + y4842 + y5277 + y5712)*z1^3, (y114 + y317 + y404 + y839 + y1013 + y1187 + y1941 + y2231 + y2521 + y2811 + y3971 + y4406 + y4841 + y5276 + y5711)*z1^2*z2, (y86 + y231 + y434 + y666 + y1043 + y1217 + y1652 + y2261 + y2551 + y2841 + y3537 + y4436 + y4871 + y5306 + y5741)*z1*z2^2, (y2 + y85 + y230 + y433 + y665 + y1042 + y1216 + y1651 + y2260 + y2550 + y2840 + y3536 + y4435 + y4870 + y5305 + y5740)*z2^3, (y113 + y316 + y403 + y838 + y1012 + y1186 + y1940 + y2230 + y2520 + y2810 + y3970 + y4405 + y4840 + y5275 + y5710)*z1^2*z3, (y84 + y229 + y432 + y664 + y1041 + y1215 + y1650 + y2259 + y2549 + y2839 + y3535 + y4434 + y4869 + y5304 + y5739)*z2^2*z3, (y57 + y144 + y463 + y492 + y1072 + y1246 + y1362 + y2290 + y2580 + y2870 + y3102 + y4465 + y4900 + y5335 + y5770)*z1*z3^2, (y56 + y143 + y462 + y491 + y1071 + y1245 + y1361 + y2289 + y2579 + y2869 + y3

z9, (y310 + y318 + y347 + y376 + y608 + y782 + y1159 + y1478 + y1768 + y2377 + y3044 + y3218 + y3653 + y4552 + y5509 + y5944)*z1*z9^2, (y317 + y346 + y375 + y607 + y781 + y1158 + y1477 + y1767 + y2376 + y3043 + y3217 + y3652 + y4551 + y5508 + y5943)*z2*z9^2, (y316 + y345 + y374 + y606 + y780 + y1157 + y1476 + y1766 + y2375 + y3042 + y3216 + y3651 + y4550 + y5507 + y5942)*z3*z9^2, (y315 + y339 + y344 + y373 + y605 + y779 + y1156 + y1475 + y1765 + y2374 + y3041 + y3215 + y3650 + y4549 + y5506 + y5941)*z4*z9^2, (y314 + y343 + y368 + y372 + y604 + y778 + y1155 + y1474 + y1764 + y2373 + y3040 + y3214 + y3649 + y4548 + y5505 + y5940)*z5*z9^2, (y313 + y342 + y371 + y603 + y777 + y1154 + y1473 + y1763 + y2372 + y3039 + y3213 + y3648 + y4547 + y5504 + y5939)*z6*z9^2, (y312 + y341 + y370 + y602 + y776 + y1153 + y1472 + y1762 + y2371 + y3038 + y3212 + y3647 + y4546 + y5503 + y5938)*z7*z9^2, (y311 + y340 + y369 + y601 + y775 + y1152 + y1471 + y1761 + y2370 + y3037 + y3211 + y3646 + y4545 + y5502 +

y4610 + y5045 + y6031)*z1*z12^2, (y655 + y665 + y694 + y723 + y752 + y781 + y810 + y1535 + y2144 + y2434 + y2724 + y3275 + y4174 + y4609 + y5044 + y6030)*z2*z12^2, (y664 + y693 + y722 + y751 + y780 + y809 + y1534 + y2143 + y2433 + y2723 + y3274 + y4173 + y4608 + y5043 + y6029)*z3*z12^2, (y663 + y684 + y692 + y721 + y750 + y779 + y808 + y1533 + y2142 + y2432 + y2722 + y3273 + y4172 + y4607 + y5042 + y6028)*z4*z12^2, (y662 + y691 + y720 + y749 + y778 + y807 + y1532 + y2141 + y2431 + y2721 + y3272 + y4171 + y4606 + y5041 + y6027)*z5*z12^2, (y661 + y690 + y713 + y719 + y748 + y777 + y806 + y1531 + y2140 + y2430 + y2720 + y3271 + y4170 + y4605 + y5040 + y6026)*z6*z12^2, (y660 + y689 + y718 + y742 + y747 + y776 + y805 + y1530 + y2139 + y2429 + y2719 + y3270 + y4169 + y4604 + y5039 + y6025)*z7*z12^2, (y659 + y688 + y717 + y746 + y775 + y804 + y1529 + y2138 + y2428 + y2718 + y3269 + y4168 + y4603 + y5038 + y6024)*z8*z12^2, (y658 + y687 + y716 + y745 + y771 + y774 + y803 + y1528 + y2137 + y2427

 + y3326 + y3761 + y4196 + y5095 + y6081)*z9*z14^2, (y1005 + y1034 + y1063 + y1092 + y1121 + y1150 + y1585 + y1875 + y2165 + y2774 + y3325 + y3760 + y4195 + y5094 + y6080)*z10*z14^2, (y1004 + y1033 + y1062 + y1091 + y1120 + y1149 + y1584 + y1874 + y2164 + y2773 + y3324 + y3759 + y4194 + y5093 + y6079)*z11*z14^2, (y1003 + y1032 + y1061 + y1090 + y1119 + y1148 + y1583 + y1873 + y2163 + y2772 + y3323 + y3758 + y4193 + y5092 + y6078)*z12*z14^2, (y1002 + y1031 + y1060 + y1089 + y1118 + y1147 + y1582 + y1872 + y2162 + y2771 + y3322 + y3757 + y4192 + y5091 + y6077)*z13*z14^2, (y14 + y1001 + y1030 + y1059 + y1088 + y1117 + y1146 + y1581 + y1871 + y2161 + y2770 + y3321 + y3756 + y4191 + y5090 + y6076)*z14^3, (y101 + y304 + y391 + y826 + y1000 + y1174 + y1188 + y1928 + y2218 + y2508 + y2798 + y3958 + y4393 + y4828 + y5263 + y5698)*z1^2*z15, (y1187 + y1217)*z1*z2*z15, (y72 + y217 + y420 + y652 + y1029 + y1203 + y1216 + y1638 + y2247 + y2537 + y2827 + y3523 + y4422 + y4857 + y5292 + y5727)*z2^2*z1

y1621)*z3*z15*z16, (y1260 + y1620)*z4*z15*z16, (y1289 + y1377 + y1619)*z5*z15*z16, (y1318 + y1406 + y1618)*z6*z15*z16, y1617*z7*z15*z16, (y1435 + y1616)*z8*z15*z16, (y1464 + y1615)*z9*z15*z16, (y1493 + y1614)*z10*z15*z16, y1613*z11*z15*z16, (y1522 + y1612)*z12*z15*z16, (y1551 + y1611)*z13*z15*z16, (y1580 + y1610)*z14*z15*z16, (y1173 + y1202 + y1231 + y1260 + y1289 + y1318 + y1608 + y1609 + y1898 + y2188 + y2478 + y3348 + y3783 + y4218 + y4653 + y6103)*z15^2*z16, (y1362 + y1391 + y1420 + y1449 + y1478 + y1507 + y1536 + y1565 + y1594 + y1623 + y3827 + y4262 + y4697 + y5132 + y5567)*z1*z16^2, (y1361 + y1390 + y1419 + y1448 + y1477 + y1506 + y1535 + y1564 + y1593 + y1622 + y3826 + y4261 + y4696 + y5131 + y5566)*z2*z16^2, (y1347 + y1360 + y1389 + y1418 + y1447 + y1476 + y1505 + y1534 + y1563 + y1592 + y1621 + y3825 + y4260 + y4695 + y5130 + y5565)*z3*z16^2, (y1359 + y1388 + y1417 + y1446 + y1475 + y1504 + y1533 + y1562 + y1591 + y1620 + y3824 + y4259 + y4694 + y5129 + y5564)*z4*z16^2, (y135

...

y31 + y118 + y437 + y466 + y1046 + y1220 + y1336 + y2264 + y2554 + y2844 + y3076 + y4439 + y4874 + y5309 + y5333 + y5744)*z3^2*z27, (y5274 + y5364)*z1*z4*z27, (y5303 + y5363)*z2*z4*z27, (y31 + y5332 + y5362)*z3*z4*z27, (y31 + y234 + y321 + y669 + y843 + y1249 + y1655 + y1945 + y2583 + y2873 + y3540 + y3975 + y4903 + y5338 + y5361 + y5773)*z4^2*z27, (y5273 + y5393)*z1*z5*z27, (y60 + y5302 + y5392)*z2*z5*z27, (y5331 + y5391)*z3*z5*z27, (y5360 + y5390)*z4*z5*z27, (y60 + y147 + y350 + y495 + y872 + y1278 + y1365 + y1974 + y2612 + y2902 + y3105 + y4004 + y4932 + y5367 + y5389 + y5802)*z5^2*z27, (y89 + y5272 + y5422)*z1*z6*z27, (y5301 + y5421)*z2*z6*z27, (y5330 + y5420)*z3*z6*z27, (y5359 + y5419)*z4*z6*z27, (y5388 + y5418)*z5*z6*z27, (y89 + y176 + y263 + y524 + y698 + y1307 + y1394 + y1684 + y2641 + y2931 + y3134 + y3569 + y4961 + y5396 + y5417 + y5831)*z6^2*z27, (y5271 + y5451)*z1*z7*z27, (y5300 + y5450)*z2*z7*z27, (y118 + y5329 + y5449)*z3*z7*z27, (y5358 + y5448)*z4*z7*z27, (y147 + y5387 +

y4468 + y4497 + y4526 + y4555 + y4584 + y4613 + y4642 + y4671 + y4700 + y4729 + y4758 + y4787)*z25^2*z27, (y4816 + y5252)*z1*z26*z27, (y4845 + y5281)*z2*z26*z27, (y4874 + y5310)*z3*z26*z27, (y4903 + y5339)*z4*z26*z27, (y4932 + y5368)*z5*z26*z27, (y4961 + y5397)*z6*z26*z27, y5426*z7*z26*z27, y5455*z8*z26*z27, y5484*z9*z26*z27, y5513*z10*z26*z27, y4990*z11*z26*z27, y5019*z12*z26*z27, y5048*z13*z26*z27, y5077*z14*z26*z27, (y5106 + y5542)*z16*z26*z27, (y5135 + y5571)*z17*z26*z27, (y5164 + y5600)*z18*z26*z27, (y5193 + y5629)*z19*z26*z27, y5658*z20*z26*z27, y5222*z21*z26*z27, (y4816 + y4845 + y4874 + y4903 + y4932 + y4961 + y4990 + y5019 + y5048 + y5077 + y5106 + y5135 + y5164 + y5193 + y5222)*z26^2*z27, (y5251 + y5277 + y5306 + y5335 + y5364 + y5393 + y5422 + y5451 + y5480 + y5509 + y5538 + y5567 + y5596 + y5625 + y5654 + y5683)*z1*z27^2, (y5276 + y5280 + y5305 + y5334 + y5363 + y5392 + y5421 + y5450 + y5479 + y5508 + y5537 + y5566 + y5595 + y5624 + y5653 + y5682)*z2*z27^2, (y5275 + y5304 +

*z28, (y1712 + y5870)*z7*z17*z28, y5899*z8*z17*z28, (y1741 + y5928)*z9*z17*z28, (y1770 + y5957)*z10*z17*z28, (y1799 + y5986)*z11*z17*z28, y6015*z12*z17*z28, (y1828 + y6044)*z13*z17*z28, (y1857 + y6073)*z14*z17*z28, (y1886 + y6102)*z15*z17*z28, (y1625 + y1654 + y1683 + y1712 + y1741 + y1770 + y1799 + y1828 + y1857 + y1886 + y3365 + y4264 + y4699 + y5134 + y5569)*z17^2*z28, (y1915 + y5695)*z1*z18*z28, y5724*z2*z18*z28, y5753*z3*z18*z28, (y1944 + y5782)*z4*z18*z28, (y1973 + y5811)*z5*z18*z28, y5840*z6*z18*z28, (y2002 + y5869)*z7*z18*z28, (y2031 + y5898)*z8*z18*z28, y5927*z9*z18*z28, (y2060 + y5956)*z10*z18*z28, (y2089 + y5985)*z11*z18*z28, (y2118 + y6014)*z12*z18*z28, y6043*z13*z18*z28, (y2147 + y6072)*z14*z18*z28, (y2176 + y6101)*z15*z18*z28, (y1915 + y1944 + y1973 + y2002 + y2031 + y2060 + y2089 + y2118 + y2147 + y2176 + y3394 + y3829 + y4728 + y5163 + y5598)*z18^2*z28, (y2205 + y5694)*z1*z19*z28, (y2234 + y5723)*z2*z19*z28, (y2263 + y5752)*z3*z19*z28, y5781*z4*z19*z28, y5810*z5*z19*z28

 + y3538 + y3973 + y4901 + y5336 + y5771)*z4^2, y58*z2*z5, (y58 + y145 + y348 + y493 + y870 + y1276 + y1363 + y1972 + y2610 + y2900 + y3103 + y4002 + y4930 + y5365 + y5800)*z5^2, y87*z1*z6, (y87 + y174 + y261 + y522 + y696 + y1305 + y1392 + y1682 + y2639 + y2929 + y3132 + y3567 + y4959 + y5394 + y5829)*z6^2, y116*z3*z7, y145*z5*z7, y174*z6*z7, (y116 + y145 + y174 + y725 + y899 + y1073 + y1711 + y2001 + y2291 + y2958 + y3596 + y4031 + y4466 + y5423 + y5858)*z7^2, y203*z2*z8, y232*z4*z8, y261*z6*z8, (y203 + y232 + y261 + y551 + y928 + y1102 + y1421 + y2030 + y2320 + y2987 + y3161 + y4060 + y4495 + y5452 + y5887)*z8^2, y290*z1*z9, y319*z4*z9, y348*z5*z9, (y290 + y319 + y348 + y580 + y754 + y1131 + y1450 + y1740 + y2349 + y3016 + y3190 + y3625 + y4524 + y5481 + y5916)*z9^2, y377*z1*z10, y406*z2*z10, y435*z3*z10, (y377 + y406 + y435 + y609 + y783 + y957 + y1479 + y1769 + y2059 + y3045 + y3219 + y3654 + y4089 + y5510 + y5945)*z10^2, y464*z3*z11, y493*z5*z11, y522*z6*z11, y551*z8*z11, y580*z9

6118-element Vector{Float64}:
 0.03567190744586175
 0.03567190744586175
 0.03567190744586176
 0.035671907445861746
 0.035671907445861774
 0.035671907445861746
 0.035671907445861746
 0.035671907445861746
 0.03567190744586176
 0.03567190744586177
 ⋮
 0.00020394541679027273
 0.00020394541679026753
 0.00020394541679027783
 0.0003955305052902202
 0.0003955305052902266
 0.0003955305052902304
 0.00039553050529022443
 0.00039553050529022004
 0.0003955305052902239

In [9]:
nver, E = text_reader("Grafos/myciel7.col.txt")
g = Graph{Undirected}(nver)
for v in E
    add_edge!(g, v[1], v[2])
end

In [19]:
nulla(g,56, 3)

### 3. NulLA con simetrías
Ahora, implementaremos una de las ideas para optimizar NulLA. Particularmente, utilizaremos las simetrías que existen en el grafo. Para esto, dado un grafo de $n$ vértices, primero debemos hallar un subgrupo $G$ de $S_n$ tal que las ecuaciones polinomiales asociadas al $k$-coloreo del grafo es invariante bajo la acción del grupo $G$ (permutando los índices de las variables $x_1, \dots , x_n$). Así, definimos un par de funciones que permiten dado un grafo calcular su matriz de adyacencia y a partir de esta encontrar una permutación $ \sigma \in S_n$ tal que su matriz de permutación asociada conmute con la matriz de adyacencia del grafo, y así tomaremos $G = < \sigma >$.

In [12]:
function adjacency_matrix(n, Edges)
    adj_M = zeros(n , n)
    for e in Edges
        adj_M[e[1], e[2]] = 1
        adj_M[e[2], e[1]] = 1
    end
    return adj_M
    
end

function find_perm_group(n, Edges, m)
    M = adjacency_matrix(n, Edges)
    Sn = symmetric_group(n)
    auto = []
    for g in Sn    
        P = Matrix(permutation_matrix(QQ, g))
        if P*M == M*P
            push!(auto, g)
        end

        if length(auto) > m
            return auto[2:end]
        end
    end
    
end


find_perm_group (generic function with 1 method)

Ahora, una vez que hemos encontrado el subgrupo $G$ que buscabamos, debemos calcular las órbitas de la acción de $G$ sobre los monomios que componen el sistema de polinomios asociados al grafo.

In [13]:
function orbit(y, X, G)
    o = []
    for g in G
        for x in X
            if x^g == y
                push!(o, x)
            end
        end
    end

    return unique(o)
end
function orbits(X, G)
    orbs = []
    
    for y in X
        state = false
        for o in orbs 
            if y in o
                state = true
            end
        end
        if state == false
            push!(orbs, orbit(y, X, G))
        end
    end
    return orbs
end
       

orbits (generic function with 1 method)

Ahora, procedemos a probar lo implementado con un par de grafos.

Grafo completo $K_4$:

In [14]:
edg = [[1, 2], [1, 3], [1, 4], [2, 3], [2, 4], [3, 4]]

R, x = PolynomialRing(QQ, ["x$i" for i in 1:4]) 
listap =[x[1]^3 - 1
,x[2]^3 - 1
,x[3]^3 - 1
,x[4]^3 - 1
,x[1]^2 + x[1]*x[2] + x[2]^2
,x[1]^2 + x[1]*x[3] + x[3]^2
,x[1]^2 + x[1]*x[4] + x[4]^2
,x[2]^2 + x[2]*x[3] + x[3]^2
,x[2]^2 + x[2]*x[4] + x[4]^2
,x[3]^2 + x[3]*x[4] + x[4]^2]

10-element Vector{QQMPolyRingElem}:
 x1^3 - 1
 x2^3 - 1
 x3^3 - 1
 x4^3 - 1
 x1^2 + x1*x2 + x2^2
 x1^2 + x1*x3 + x3^2
 x1^2 + x1*x4 + x4^2
 x2^2 + x2*x3 + x3^2
 x2^2 + x2*x4 + x4^2
 x3^2 + x3*x4 + x4^2

In [15]:
graph_auto = find_perm_group(4, edg, 23)

23-element Vector{Any}:
 (3,4)
 (2,4)
 (2,4,3)
 (2,3,4)
 (2,3)
 (1,4)
 (1,4,3)
 (1,4,2)
 (1,4,3,2)
 (1,4,2,3)
 ⋮
 (1,2)(3,4)
 (1,2,3)
 (1,2,3,4)
 (1,3,4)
 (1,3)
 (1,3,4,2)
 (1,3,2)
 (1,3)(2,4)
 (1,3,2,4)

In [16]:
mono = [1
,x[1]^3
,x[1]^2*x[2]
,x[1]^2*x[3]
,x[1]^2*x[4]
,x[1]*x[2]^2
,x[1]*x[3]^2
,x[1]*x[4]^2
,x[1]*x[2]*x[3]
,x[1]*x[2]*x[4]
,x[1]*x[3]*x[4]
,x[2]^3 
,x[3]^3
,x[4]^3
,x[2]^2*x[3]
,x[3]^2*x[4]
,x[2]*x[4]^2
,x[2]^2*x[4]
,x[2]*x[3]^2
,x[3]*x[4]^2
,x[2]*x[3]*x[4]
]
orbits(mono, graph_auto)

4-element Vector{Any}:
 Any[1]
 Any[x1^3, x4^3, x3^3, x2^3]
 Any[x1^2*x2, x1^2*x4, x1^2*x3, x2*x4^2, x2*x3^2, x2^2*x4, x2^2*x3, x3^2*x4, x3*x4^2, x1*x4^2, x1*x3^2, x1*x2^2]
 Any[x1*x2*x4, x1*x3*x4, x1*x2*x3, x2*x3*x4]

In [17]:
nver, E = text_reader("Grafos/johnson8-2-4.clq.txt")
g = Graph{Undirected}(nver)
for v in E
    add_edge!(g, v[1], v[2])
end

R, x = PolynomialRing(QQ, ["x$i" for i in 1:nver]) # Definimos el anillo de polinomios con n = #vercies del grafo variables

G = polisGrafo(g, 3, R)

238-element Vector{QQMPolyRingElem}:
 x1^3 - 1
 x2^3 - 1
 x3^3 - 1
 x4^3 - 1
 x5^3 - 1
 x6^3 - 1
 x7^3 - 1
 x8^3 - 1
 x9^3 - 1
 x10^3 - 1
 ⋮
 x7^2 + x7*x28 + x28^2
 x8^2 + x8*x28 + x28^2
 x9^2 + x9*x28 + x28^2
 x10^2 + x10*x28 + x28^2
 x11^2 + x11*x28 + x28^2
 x12^2 + x12*x28 + x28^2
 x13^2 + x13*x28 + x28^2
 x14^2 + x14*x28 + x28^2
 x15^2 + x15*x28 + x28^2

In [18]:
graph_auto = find_perm_group(nver, E, 1)